In [72]:
# Dependencies
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import urllib.request
import requests
import pandas as pd
import time
import re
import json
import matplotlib.pyplot as plt
import numpy as np
import time
import warnings
from sqlalchemy import create_engine

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

# Scrape Program to gather Current Data from truecar.com

In [73]:
# Scrape Program

base_url = 'https://www.truecar.com/used-cars-for-sale/listings/body-truck/location-chicago-il/?accidentHistory=none&page='
tail = '&searchRadius=500'
page = 0
loops = 330
    
header = ['Brand','Model','Year','Trim','Drive','Description','Exterior Color','Interior Color','Miles','Price']
rows=[]
truck_data = pd.DataFrame(data=rows,columns=header)
#rowss= []
#rows1 = []
#trucks_data1 = pd.DataFrame(data=rows1,columns=header1)
    
    
for x in range(loops):
    page = x+1
    new_url = base_url+str(page)+tail
    html = requests.get(new_url)
    soup = BeautifulSoup(html.text,'lxml')
    #header = ['Brand','Model','Year','Trim','Drive','Description','Exterior Color','Interior Color','Miles','Price']

    div_list = soup.find_all('div', class_='card-content vehicle-card-body order-3')
    
    #rowss = []
    #rows = []

    for x in range(len(div_list)):

        # Start Row Addition----------------
        row = []

        # Call Truck HTML Data----------------
        truck = div_list[x]

        # Save Brand Data----------------
        try:
            brand = soup.find('span', class_='vehicle-header-make-model text-truncate').text
            num_brand= (len(brand))-(brand.find(" "))
            brand1 = brand[:-num_brand]
            row.append(brand1)
        except:
            row.append(np.nan)
            
        # Save Model Data----------------
        try:
            num_model = (brand.find(" "))+1
            model = brand[num_model:]
            row.append(model)
        except:
            row.append(np.nan)
            
        # Save Year Data----------------
        try:
            year = truck.find('span', class_='vehicle-card-year font-size-1').text
            row.append(year)
        except:
            row.append(np.nan)
            
        # Save Trim Data----------------
        try:
            desc = truck.find('div', class_='font-size-1 text-truncate').text
            num_trim = (len(desc))-(desc.find(" "))
            trim = desc[:-num_trim]
            row.append(trim)
        except:
            row.append(np.nan)
            
        # Save Drive Data----------------
        try:
            num_drive = (len(desc))-(desc.rfind(" "))-1
            drive = desc[-num_drive:]
            row.append(drive)
        except:
            row.append(np.nan)
            
        # Save Description Data----------------
        try:
            desc_mid1 = desc[(-num_trim)+1:]
            desc_mid2 = desc_mid1[:-num_drive]
            desc_mid3 = desc_mid2.strip()
            row.append(desc_mid2)
        except:
            row.append(np.nan)
            
        # Save Exterior Color Data----------------
        try:
            colors = truck.find('div',class_='vehicle-card-location font-size-1 margin-top-1 text-truncate').text
            num_col = (len(colors))-(colors.find(","))
            ext_col = colors[:-num_col]
            row.append(ext_col)
        except:
            row.append(np.nan)
            
        # Save Interior Color Data----------------
        try:
            num_inter_col = (colors.find(","))+2
            inter_col= colors[num_inter_col:]
            row.append(inter_col)
        except:
            row.append(np.nan)
            
        # Save Miles Data----------------
        try:
            miles = truck.find('div', class_='d-flex w-100 justify-content-between').text
            num_miles = (len(miles))-(miles.find(" "))
            miles1 = miles[:-num_miles]
            miles2 = re.sub('[!,]','',miles1)
            row.append(miles2)
        except:
            row.append(np.nan)
            
        # Save Price Data----------------
        try:
            price = truck.find('div', class_='heading-3 margin-y-1 font-weight-bold').text
            num_price = (len(price))-(1)
            price1 = price[-num_price:]
            price2 = re.sub('[!,]','',price1)
            row.append(price2)
        except:
            row.append(np.nan)
            
        rows.append(row)
    
truck_data = pd.DataFrame(data=rows,columns=header)

In [75]:
# Print Data Frame

truck_data.head(5)

,Brand,Model,Year,Trim,Drive,Description,Exterior Color,Interior Color,Miles,Price
0,Ford,F-150,2017,Platinum,4WD,SuperCrew 5.5' Box,Silver exterior,Black interior,36956,47499
1,Ford,F-150,2012,Laramie,4WD,"Crew Cab 5'7"" Box",White exterior,Gray interior,60579,31990
2,Ford,F-150,2019,Big,4WD,"Horn/Lone Star Crew Cab 5'7"" Box",White exterior,Black interior,39058,39997
3,Ford,F-150,2013,XLT,4WD,SuperCrew 5.5' Box,Black exterior,Gray interior,116165,19995
4,Ford,F-150,2010,XL,4WD,SuperCrew 5.5' Box,White exterior,Unknown interior,157719,13495


# Review and Transform Current Data (From Web Scrape)

In [76]:
truck_data.describe()

,Brand,Model,Year,Trim,Drive,Description,Exterior Color,Interior Color,Miles,Price
count,9903,9903,9903,9903,9903,9903,9903,9903,9903,9893
unique,8,24,26,103,56,392,16,9,9440,4382
top,Ford,F-150,2019,XLT,4WD,SuperCrew 5.5' Box,White exterior,Black interior,17883,36990
freq,3483,3003,2319,1433,7483,1857,2679,6236,4,49


In [77]:
# Drop Rows 

truck_data.dropna(axis=0, how='any', inplace=True)

In [78]:
# Adjusting the data types
truck_data['Price'] = truck_data['Price'].astype(int)
truck_data['Miles'] = truck_data['Miles'].astype(int)
truck_data['Year'] = truck_data['Year'].astype(int)

In [79]:
truck_data.dtypes

Brand             object
Model             object
Year               int32
Trim              object
Drive             object
Description       object
Exterior Color    object
Interior Color    object
Miles              int32
Price              int32
dtype: object

In [80]:
truck_data[['Miles','Price']].describe()

,Miles,Price
count,9893.000000,9893.000000
mean,50526.915091,39953.760841
std,40037.964006,12536.643550
min,11.000000,2000.000000
25%,21472.000000,32000.000000
50%,40409.000000,38999.000000
75%,68642.000000,46948.000000
max,488580.000000,105000.000000


In [81]:
truck_data = truck_data[['Brand',
                         'Model',
                         'Year',
                         'Miles',
                         'Price'
                        ]]

In [82]:
truck_data.head()

,Brand,Model,Year,Miles,Price
0,Ford,F-150,2017,36956,47499
1,Ford,F-150,2012,60579,31990
2,Ford,F-150,2019,39058,39997
3,Ford,F-150,2013,116165,19995
4,Ford,F-150,2010,157719,13495


In [83]:
# Set the Index as the ID. Will be used as primary key in database

truck_data["id"] = truck_data.index + 1

truck_data.head()

,Brand,Model,Year,Miles,Price,id
0,Ford,F-150,2017,36956,47499,1
1,Ford,F-150,2012,60579,31990,2
2,Ford,F-150,2019,39058,39997,3
3,Ford,F-150,2013,116165,19995,4
4,Ford,F-150,2010,157719,13495,5


In [84]:
# Set index
truck_data.set_index("id", inplace=True)

truck_data.head()

,Brand,Model,Year,Miles,Price
id,,,,,
1,Ford,F-150,2017,36956,47499
2,Ford,F-150,2012,60579,31990
3,Ford,F-150,2019,39058,39997
4,Ford,F-150,2013,116165,19995
5,Ford,F-150,2010,157719,13495


In [101]:
# Rename the column headers to match the format of the SQL table

truck_data = truck_data.rename(columns={'Brand':'brand',
                                 'Model':'model',
                                 'Year':'year',
                                 'Miles':'miles',
                                 'Price':'price',}
          )

truck_data.head()

,brand,model,year,miles,price
id,,,,,
1,Ford,F-150,2017,36956,47499
2,Ford,F-150,2012,60579,31990
3,Ford,F-150,2019,39058,39997
4,Ford,F-150,2013,116165,19995
5,Ford,F-150,2010,157719,13495


In [102]:
truck_data.to_csv('truck_data12.csv',index=False,header=True)
print('File_Saved')

File_Saved


# Review and Transform Data From Kaggle (Historical 2018-2020)

In [87]:
file = 'Pickup_Data_Clean.csv'
df_hist = pd.read_csv(file)
df_hist.head()

,brandName,modelName,vf_ModelID,vf_ModelYear,mileage,askPrice,msrp,color,vf_EngineCylinders,vf_FuelTypePrimary,vin
0,FORD,F-250,1805.0,2017.0,19106,40962,40962,Ingot Silver Metallic,8.0,Diesel,bcd4bbecbba3b9ed2330d543abc3d90b997aa8fa6e628d...
1,FORD,F-150,1801.0,2017.0,28194,28962,28962,Ingot Silver Metallic,6.0,Gasoline,a8f09496d9523e33ee3a8f885a475f5ef001d68a334c98...
2,GMC,Sierra,1857.0,2017.0,7483,43962,43962,Crimson Red Tintcoat,8.0,Gasoline,c65dd3e96f4f68a0d4c7b09005273e0312295057c7c529...
3,FORD,F-250,1805.0,2017.0,23881,38962,38962,Oxford White,8.0,Diesel,966168515802a7cf510d6c87a15cf71cc4385316a504c6...
4,GMC,Canyon,4093.0,2017.0,6333,30962,30962,Onyx Black,6.0,Gasoline,9c7648c843035d0b46e9d716d947dfa3d0e8881054247a...


In [88]:
df_hist.describe()

,vf_ModelID,vf_ModelYear,mileage,askPrice,msrp,vf_EngineCylinders
count,50788.000000,50788.000000,50788.000000,50788.000000,5.078800e+04,50788.000000
mean,4210.843802,2014.094983,60745.412322,26767.678743,2.853393e+04,7.223951
std,4726.015433,4.001865,46278.783194,9963.884567,4.903721e+04,1.071533
min,1801.000000,1987.000000,1000.000000,5010.000000,5.010000e+03,4.000000
25%,1801.000000,2012.000000,25372.500000,19995.000000,2.099800e+04,6.000000
50%,1850.000000,2015.000000,46684.000000,27495.000000,2.899000e+04,8.000000
75%,2467.000000,2017.000000,86193.250000,32990.000000,3.476175e+04,8.000000
max,25919.000000,2020.000000,249913.000000,54999.000000,1.000000e+07,10.000000


In [89]:
# Drop Columns

df_hist = df_hist[['brandName',
                 'modelName',
                 'vf_ModelYear',
                 'mileage',
                 'askPrice'
                      ]]

In [90]:
# Rename Columns

df_hist = df_hist.rename(columns={'brandName':'brand',
                                 'modelName':'model',
                                 'vf_ModelYear':'year',
                                 'mileage':'miles',
                                 'askPrice':'price',}
          )

In [91]:
df_hist['year'] = df_hist['year'].astype(int)

In [92]:
df_hist.dtypes

brand    object
model    object
year      int32
miles     int64
price     int64
dtype: object

In [93]:
df_hist.head()

,brand,model,year,miles,price
0,FORD,F-250,2017,19106,40962
1,FORD,F-150,2017,28194,28962
2,GMC,Sierra,2017,7483,43962
3,FORD,F-250,2017,23881,38962
4,GMC,Canyon,2017,6333,30962


In [94]:
# Add an id column (the index)

df_hist["id"] = df_hist.index + 1

df_hist.head()

,brand,model,year,miles,price,id
0,FORD,F-250,2017,19106,40962,1
1,FORD,F-150,2017,28194,28962,2
2,GMC,Sierra,2017,7483,43962,3
3,FORD,F-250,2017,23881,38962,4
4,GMC,Canyon,2017,6333,30962,5


In [95]:
# Set index
df_hist.set_index("id", inplace=True)

df_hist.head()

,brand,model,year,miles,price
id,,,,,
1,FORD,F-250,2017,19106,40962
2,FORD,F-150,2017,28194,28962
3,GMC,Sierra,2017,7483,43962
4,FORD,F-250,2017,23881,38962
5,GMC,Canyon,2017,6333,30962


# Create the Database Connection

In [96]:
connection_string = "postgres:Rubenast09!@localhost:5432/vehicle_data"
engine = create_engine(f'postgresql://{connection_string}')

In [97]:
# Confirm tables
engine.table_names()

['historical_data', 'current_data']

# Load DataFrames into database

In [103]:
truck_data.to_sql(name='current_data', con=engine, if_exists='append', index=True)

In [104]:
#Check to make sure that the data has been added by querying the current_data table

pd.read_sql_query('select * from current_data', con=engine).head()

,id,brand,model,year,miles,price
0,1,Ford,F-150,2017,36956,47499
1,2,Ford,F-150,2012,60579,31990
2,3,Ford,F-150,2019,39058,39997
3,4,Ford,F-150,2013,116165,19995
4,5,Ford,F-150,2010,157719,13495


In [99]:
df_hist.to_sql(name='historical_data', con=engine, if_exists='append', index=True)

In [100]:
#Check to make sure that the data has been added by querying the historical_data table

pd.read_sql_query('select * from historical_data', con=engine).head()

,id,brand,model,year,miles,price
0,1,FORD,F-250,2017,19106,40962
1,2,FORD,F-150,2017,28194,28962
2,3,GMC,Sierra,2017,7483,43962
3,4,FORD,F-250,2017,23881,38962
4,5,GMC,Canyon,2017,6333,30962
